### test for plotting pft level data on h1 files

In [1]:
import os, sys
import shutil
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import xarray as xr
import xesmf as xe

# Helpful for plotting only
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import uxarray as ux  #need npl 2024a or later
import geoviews.feature as gf

#sys.path.append('/glade/u/home/wwieder/python/adf/lib/plotting_functions.py')
from plotting_functions import *

In [2]:
# Load datataset  
# TODO, develop function for this too
gppfile='/glade/derecho/scratch/wwieder/ADF/b.e30_beta04.BLT1850.ne30_t232_wgx3.121/climo/b.e30_beta04.BLT1850.ne30_t232_wgx3.121_GPP_climo.nc'
laih1file='/glade/derecho/scratch/wwieder/ctsm53n04ctsm52028_ne30pg3t232_hist.clm2.h1.TLAI.1860s.nc'
mesh0 = '/glade/campaign/cesm/cesmdata/inputdata/share/meshes/ne30pg3_ESMFmesh_cdf5_c20211018.nc'
#ux file for plotting
ds0 = ux.open_dataset(mesh0, gppfile)
ds1 = ux.open_dataset(mesh0, laih1file)

#xr files for manipulations
ds0b = xr.open_dataset(gppfile, decode_times=True)
ds = xr.open_dataset(laih1file, decode_times=True)

In [3]:
# Missing coords for lndgrid, add them here and change name
ds0b = ds0b.rename({'lndgrid': 'n_face'})
ds0b['n_face'] = np.arange(1,(ds.pfts1d_ixy.values.max().astype(int)+1))
ds

<xarray.Dataset> Size: 421MB
Dimensions:             (time: 120, hist_interval: 2, pft: 710683)
Coordinates:
  * time                (time) object 960B 1860-02-01 00:00:00 ... 1870-01-01...
Dimensions without coordinates: hist_interval, pft
Data variables: (12/16)
    time_bounds         (time, hist_interval) object 2kB ...
    pfts1d_lon          (pft) float64 6MB ...
    pfts1d_lat          (pft) float64 6MB ...
    pfts1d_ixy          (pft) float64 6MB 737.0 737.0 ... 4.86e+04 4.86e+04
    pfts1d_jxy          (pft) float64 6MB ...
    pfts1d_gi           (pft) float64 6MB ...
    ...                  ...
    pfts1d_wtcol        (pft) float64 6MB ...
    pfts1d_itype_veg    (pft) float64 6MB ...
    pfts1d_itype_col    (pft) float64 6MB ...
    pfts1d_itype_lunit  (pft) float64 6MB ...
    pfts1d_active       (pft) float64 6MB ...
    TLAI                (time, pft) float32 341MB ...
Attributes: (12/102)
    title:                                CLM History file information
    comment:                              NOTE: None of the variables are wei...
    Conventions:                          CF-1.0
    history:                              Thu Mar  6 06:04:05 2025: ncrcat -v...
    source:                               Community Terrestrial Systems Model
    hostname:                             derecho
    ...                                   ...
    cft_tropical_soybean:                 63
    cft_irrigated_tropical_soybean:       64
    time_period_freq:                     month_1
    Time_constant_3Dvars_filename:        ./ctsm53n04ctsm52028_ne30pg3t232_hi...
    Time_constant_3Dvars:                 ZSOI:DZSOI:WATSAT:SUCSAT:BSW:HKSAT:...
    NCO:                                  netCDF Operators version 5.2.4 (Hom...

In [5]:
# select a single PFT
## TODO this step is kind of a memory hog
npft=3
var='TLAI'
for i in range(1,npft):
    print('starting pft '+str(i))
    temp = ds.where(ds.pfts1d_itype_veg==i, drop=True).max('time')
    # TODO, this should be time evolving, but not currently doen
    # Rename coord, since the pft dimension is not meaningful
    temp= temp.rename({'pft': 'n_face'})
    #temp[var]= temp[var].rename({'pft': 'n_face'})
    #temp['pfts1d_wtgcell'].rename({'pft': 'n_face'})

    # assign values from pfts1d_ixy to n_face
    temp['n_face'] = temp.pfts1d_ixy.values.astype(int)
    temp.assign_coords({"npft": i})
    # combine along PFT variable
    if i == 1:
        dsOut = temp
    else:
        dsOut = xr.concat([dsOut, temp], dim="npft")
dsOut

starting pft 1
starting pft 2


<xarray.Dataset> Size: 4MB
Dimensions:             (npft: 2, hist_interval: 2, n_face: 15962)
Coordinates:
  * n_face              (n_face) int64 128kB 737 738 745 ... 48598 48599 48600
Dimensions without coordinates: npft, hist_interval
Data variables: (12/16)
    time_bounds         (npft, hist_interval, n_face) object 511kB 1869-12-01...
    pfts1d_lon          (npft, n_face) float64 255kB 19.5 20.5 ... 136.0 135.0
    pfts1d_lat          (npft, n_face) float64 255kB -34.9 -34.73 ... 36.2 35.74
    pfts1d_ixy          (npft, n_face) float64 255kB 737.0 738.0 ... 4.86e+04
    pfts1d_jxy          (npft, n_face) float64 255kB 1.0 1.0 1.0 ... 1.0 1.0 1.0
    pfts1d_gi           (npft, n_face) float64 255kB 1.0 2.0 ... 1.596e+04
    ...                  ...
    pfts1d_wtcol        (npft, n_face) float64 255kB 0.0 0.0 0.0 ... 0.0 0.0 0.0
    pfts1d_itype_veg    (npft, n_face) float64 255kB 1.0 1.0 1.0 ... 2.0 2.0 2.0
    pfts1d_itype_col    (npft, n_face) float64 255kB 1.0 1.0 1.0 ... 1.0 1.0 1.0
    pfts1d_itype_lunit  (npft, n_face) float64 255kB 1.0 1.0 1.0 ... 1.0 1.0 1.0
    pfts1d_active       (npft, n_face) float64 255kB nan nan nan ... nan nan nan
    TLAI                (npft, n_face) float32 128kB nan nan nan ... nan nan nan

In [6]:
# align subset pft output with plotting data array
target = ds0b.GPP.isel(time=0)
AlignOut, target = xr.align(dsOut, target, join="right")

In [7]:
dsplot = ds0.max('time')
dsplot[var] = AlignOut[var]
dsplot['pfts1d_wtgcell'] = AlignOut['pfts1d_wtgcell']
dsplot

<xarray.UxDataset> Size: 3MB
Dimensions:         (n_face: 48600, npft: 2)
Coordinates:
  * n_face          (n_face) int64 389kB 1 2 3 4 5 ... 48597 48598 48599 48600
Dimensions without coordinates: npft
Data variables:
    GPP             (n_face) float32 194kB nan nan nan ... 7.436e-05 8.893e-05
    area            (n_face) float32 194kB nan nan nan ... 9.519e+03 9.519e+03
    landfrac        (n_face) float32 194kB nan nan nan ... 0.6608 0.2991 0.07713
    landmask        (n_face) float64 389kB nan nan nan nan ... nan 1.0 1.0 1.0
    TLAI            (npft, n_face) float32 389kB nan nan nan nan ... nan nan nan
    pfts1d_wtgcell  (npft, n_face) float64 778kB nan nan nan nan ... 0.0 0.0 0.0

In [8]:
#var='pfts1d_wtgcell'
var='TLAI'
transform = ccrs.PlateCarree()
proj = ccrs.PlateCarree()
cmap = plt.cm.viridis_r
cmap.set_under(color='deeppink')
levels = [0.1, 1, 2, 3, 4, 5, 6,7]

figsize= (6, 3)

# create figure object
fig, axs = plt.subplots(2,1,
    figsize=figsize,
    facecolor="w",
    constrained_layout=True,
    subplot_kw=dict(projection=proj),
    #**cp_info['subplots_opt']
)
axs=axs.flatten()


for i in range(2):
    wgts = dsplot.area + dsplot.landfrac * dsplot.pfts1d_wtgcell.isel(npft=i)
    wgts = wgts / wgts.sum()
    print(np.round((dsplot[var].isel(npft=i)*wgts).sum().values,2))
    ac = dsplot[var].isel(npft=i).to_polycollection(projection=proj)
    ac.set_cmap(cmap)
    ac.set_antialiased(False)
    ac.set_transform(transform)
    ac.set_clim(vmin=0.1,vmax=7)
    axs[i].add_collection(ac)
    cbar = plt.colorbar(ac, ax=axs[i], orientation='vertical', pad=0.05, shrink=0.8)
    axs[i].set_title('Max LAI, PFT= '+str(i+1), loc='left')

    for a in axs:
        a.coastlines()
        a.set_global()
        a.spines['geo'].set_linewidth(1.) #cartopy's recommended method
        a.set_extent([-180, 180, -60, 86])
        #a.set_xticks(np.linspace(-180, 120, 6), crs=proj)
        #a.set_yticks(np.linspace(-90, 90, 7), crs=proj)
        #a.tick_params('both', length=5, width=1.5, which='major')
        #a.tick_params('both', length=5, width=1.5, which='minor')
        #a.xaxis.set_major_formatter(lon_formatter)
        #a.yaxis.set_major_formatter(lat_formatter)

fig.savefig('h1_test', bbox_inches='tight', dpi=300)
print('-- wrote pft '+var+' figure --')

0.92
0.22
-- wrote pft TLAI figure --


/glade/derecho/scratch/wwieder/testPolarFig.png made
